# Grammar Fix Bot

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import Dataset
import torch
import sqlite3

c:\Users\davyd\miniconda3\envs\PYTHON3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Check if CUDA is available (i.e., if you have a compatible GPU)
if torch.cuda.is_available():
    print("CUDA is available. GPU will be used.")
else:
    print("CUDA is not available. Training will use CPU.")

CUDA is available. GPU will be used.


In [ ]:
import sys
print(sys.version)

3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]


## Data preparation

#### Read scraped sentences

In [ ]:
db_path = 'parser/sentences.db'
connection = sqlite3.connect(db_path)
cursor = connection.cursor()
cursor.execute("SELECT * FROM sentences;")
sentences = cursor.fetchall()
df = pd.DataFrame(sentences, columns=['id', 'Original', 'Altered', 'category'])
df = df.drop(['id','category'], axis=1)

In [ ]:
df.rename(columns={'Original': 'target', 'Altered': 'input'}, inplace=True)
df = df[['input', 'target']]
df.head()

,input,target
0,"You may copy it, give it away or re-use it und...","You may copy it, give it away or re-use it und..."
1,"Down the Rabbit-Hole CHAPETR II, The Pool of ...","Down the Rabbit-Hole CHAPTER II,The Pool of T..."
2,"In another moment down went Alice after it, ne...","In another moment down went Alice after it, ne..."
3,After a time she herd a little pattering of fe...,After a time she heard a little pattering of f...
4,I wish I hadn't cried so much!,I wish I hadn’t cried so much!


## Data preprocessing

In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
# https://huggingface.co/pszemraj/grammar-synthesis-small
# Load the tokenizer and model
model_path = "grammar-synthesis-small"  # Path from local folder
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
# Preprocess the data
def preprocess_function(examples):
    inputs = tokenizer(examples["input"], truncation=True, padding="max_length", max_length=128)
    targets = tokenizer(examples["target"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/80938 [00:00<?, ? examples/s]

Map: 100%|██████████| 80938/80938 [00:08<00:00, 9145.74 examples/s] 


In [ ]:
# Split into training and validation sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

## Build a model

In [ ]:
import wandb
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    run_name="my_custom_run_name", 
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=500,
)

c:\Users\davyd\miniconda3\envs\PYTHON3.10\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

C:\Users\davyd\AppData\Local\Temp\ipykernel_28936\2976303973.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Fine-tune the model
trainer.train()

  4%|▎         | 500/13659 [06:45<2:55:38,  1.25it/s]

{'loss': 4.5828, 'grad_norm': 0.3901728093624115, 'learning_rate': 4.816970495643898e-05, 'epoch': 0.11}


  7%|▋         | 1000/13659 [13:28<2:50:33,  1.24it/s]

{'loss': 0.2211, 'grad_norm': 0.4652789533138275, 'learning_rate': 4.633940991287796e-05, 'epoch': 0.22}


 11%|█         | 1500/13659 [20:14<2:43:19,  1.24it/s]

...

 99%|█████████▉| 13500/13659 [3:02:54<02:07,  1.25it/s]

{'loss': 0.1557, 'grad_norm': 0.26778683066368103, 'learning_rate': 5.82033823852405e-07, 'epoch': 2.97}


                                                       
100%|██████████| 13659/13659 [3:06:02<00:00,  1.22it/s]

{'eval_loss': 0.14532490074634552, 'eval_runtime': 59.0261, 'eval_samples_per_second': 137.126, 'eval_steps_per_second': 8.572, 'epoch': 3.0}
{'train_runtime': 11162.3307, 'train_samples_per_second': 19.578, 'train_steps_per_second': 1.224, 'train_loss': 0.3328577863130626, 'epoch': 3.0}


TrainOutput(global_step=13659, training_loss=0.3328577863130626, metrics={'train_runtime': 11162.3307, 'train_samples_per_second': 19.578, 'train_steps_per_second': 1.224, 'total_flos': 1.0155755943493632e+16, 'train_loss': 0.3328577863130626, 'epoch': 3.0})

## Evaluate model on random sentence

In [ ]:
# Example input text
text = "I has an bad grammar in this sentense."
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
outputs = model.generate(**inputs)
corrected_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Corrected Sentence:", corrected_sentence)

Corrected Sentence: I have an bad grammar in this sentence.
